In [1]:
# Import needed dependencies
import requests
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import re
from datetime import date
from bs4 import BeautifulSoup, Comment

In [2]:
team_codes_df = pd.read_csv('team_codes.csv')
games_attended_df = pd.read_csv('sports_attendance.csv')

games_attended_df["URL_Date"] = " "
games_attended_df["Year"] = " "

games_attended_df['Year'] = pd.to_datetime(games_attended_df.Date)
games_attended_df['Year'] = games_attended_df['Year'].dt.strftime('%Y')
games_attended_df['URL_Date'] = pd.to_datetime(games_attended_df.Date)
games_attended_df['URL_Date'] = games_attended_df['URL_Date'].dt.strftime('%Y%m%d0')

nba_attendance_df = games_attended_df[games_attended_df.Sport == ('NBA')].copy()
mlb_attendance_df = games_attended_df[games_attended_df.Sport == ('MLB')].copy()
nfl_attendance_df = games_attended_df[games_attended_df.Sport == ('NFL')].copy()

nba_df = nba_attendance_df.merge(team_codes_df, how='left', left_on='Home', right_on='NBA_Teams')
nba_df = nba_df.drop(columns=['MLB_Teams', 'MLB_Codes', 'NBA_Teams', 'NFL_Teams', 'NFL_Codes', 'NHL_Teams', 'NHL_Codes', 'MLS_Codes'])

nba_df['URL_Variable'] = nba_df['URL_Date'] + nba_df['NBA_Codes']

away_code_df = nba_attendance_df.merge(team_codes_df, how='left', left_on='Visitor', right_on='NBA_Teams')
away_code_df = away_code_df.drop(columns=['MLB_Teams', 'MLB_Codes', 'NBA_Teams', 'NFL_Teams', 'NFL_Codes', 'NHL_Teams', 'NHL_Codes', 'MLS_Codes'])

nba_df['Away_Code'] = away_code_df['NBA_Codes']
nba_df.rename(columns = {'NBA_Codes':'Home_Code'}, inplace = True)
nba_df


,Date,Home,Visitor,Sport,URL_Date,Year,Home_Code,URL_Variable,Away_Code
0,2/11/2014,Los Angeles Lakers,Utah Jazz,NBA,201402110,2014,LAL,201402110LAL,UTA
1,1/2/2015,Utah Jazz,Atlanta Hawks,NBA,201501020,2015,UTA,201501020UTA,ATL
2,4/3/2016,Orlando Magic,Memphis Grizzlies,NBA,201604030,2016,ORL,201604030ORL,MEM
3,12/27/2017,Golden State Warriors,Utah Jazz,NBA,201712270,2017,GSW,201712270GSW,UTA
4,11/17/2018,Orlando Magic,Los Angeles Lakers,NBA,201811170,2018,ORL,201811170ORL,LAL
5,11/18/2018,Orlando Magic,New York Knicks,NBA,201811180,2018,ORL,201811180ORL,NYK
6,12/2/2018,Miami Heat,Utah Jazz,NBA,201812020,2018,MIA,201812020MIA,UTA
7,11/8/2019,Atlanta Hawks,Sacramento Kings,NBA,201911080,2019,ATL,201911080ATL,SAC
8,11/3/2021,Sacramento Kings,New Orleans Pelicans,NBA,202111030,2021,SAC,202111030SAC,NOP
9,11/7/2021,Sacramento Kings,Indiana Pacers,NBA,202111070,2021,SAC,202111070SAC,IND


In [3]:
URL_list = nba_df["URL_Variable"].values.tolist()
Home_code_list = nba_df["Home_Code"].values.tolist()
Away_code_list = nba_df["Away_Code"].values.tolist()
Year_list = nba_df["Year"].values.tolist()
Date_list = nba_df["URL_Date"].values.tolist()
Actual_date_list = nba_df["Date"].values.tolist()

In [4]:
# soup

In [5]:
# getting length of list
length = len(URL_list)

dataframe_dictionary = {}

box_score_df_list = []
home_stats_df_list = []
away_stats_df_list = []
game_logistics_df_list = []

for i in range(length):
    
    # Create empty lists to hold table data to be scraped
    away_box_score = []
    home_box_score = []
    box_score = []
    
    # input URL and use BeautifulSoup to parse through the page
    url = f'https://www.basketball-reference.com/boxscores/{URL_list[i]}.html'
    soup = BeautifulSoup(requests.get(url).content, 'html.parser')

    # Grab the table element that has away team statistics
    away_table = soup.select_one(f'#div_box-{Away_code_list[i]}-game-basic') 

    # Grab data from table and put it into the list created above
    for tr in away_table.select('tr:has(td)'):
        tds = [td.get_text(strip=True) for td in tr.select('td')]
        away_box_score.append(tds)

    # Grab the table element that has home team statistics
    home_table = soup.select_one(f'#div_box-{Home_code_list[i]}-game-basic') 

    # Grab data from table and put it into the list created above
    for tr in home_table.select('tr:has(td)'):
        tds2 = [td.get_text(strip=True) for td in tr.select('td')]
        home_box_score.append(tds2)

    # Grab the table element that has game box score
    box_score_table = soup.select_one('#all_line_score')

    # Game box score is commented out in html, so this will grab it out of the comments
    for comment in box_score_table.find_all(text=lambda text: isinstance(text, Comment)):
        if comment.find("<table ") > 0:
            comment_soup = BeautifulSoup(comment, 'html.parser')
            table = comment_soup.find("table")

    # Grab data from table and put it into the list created above
    for tr in table.select('tr:has(td)'):
        tds3 = [td.get_text(strip=True) for td in tr.select('td')]
        box_score.append(tds3)
       
    # Create dataframe for away team statistics
    dataframe_dictionary[Away_code_list[i] + Date_list[i]] = pd.DataFrame(away_box_score)

    # Create an empty list to store away team statistics header information
    away_header_list = []

    # Grab the table header information to use as column headers in our away team statistics dataframe
    for tr in away_table.select('tr:has(th)'):
        ths = [th.get_text(strip=True) for th in tr.select('th')]
        away_header_list.append(ths)

    # Create dataframe for home team statistics
    dataframe_dictionary[Home_code_list[i] + Date_list[i]] = pd.DataFrame(home_box_score)

    # Create an empty list to store home team statistics header information
    home_header_list = []

    # Grab the table header information to use as column headers in our home team statistics dataframe
    for tr in home_table.select('tr:has(th)'):
        ths2 = [th.get_text(strip=True) for th in tr.select('th')]
        home_header_list.append(ths2)    
        
    # Create dataframe for game box score
    dataframe_dictionary['Box' + Date_list[i]] = pd.DataFrame(box_score)

    # Create an empty list to store game box score dataframe header information
    box_score_list = ['1Q', '2Q', '3Q', '4Q', 'T']

    # If a game goes into overtime(s), we need to add to the box score list for each overtime period
    OT_counter = 1
    while len(box_score[0]) > len(box_score_list):
        box_score_list.insert(-1, f'{OT_counter}OT')
        OT_counter = OT_counter + 1
        
    # Update the column and row labels
    dataframe_dictionary['Box' + Date_list[i]] = dataframe_dictionary['Box' + Date_list[i]].set_axis(box_score_list, axis=1, inplace=False)
    dataframe_dictionary['Box' + Date_list[i]] = dataframe_dictionary['Box' + Date_list[i]].set_axis([f'{Away_code_list[i]}', f'{Home_code_list[i]}'])
    
    dataframe_dictionary['Box' + Date_list[i]]['Date'] = Actual_date_list[i]
    box_score_df_list.append(dataframe_dictionary['Box' + Date_list[i]])
    
    # Pull the away team player data and combine the lists into one list
    away_starters = away_header_list[2:7]
    away_bench = away_header_list[8:-1]
    away_players = [away_starters,away_bench]

    # Pull the home team player data and combine the lists into one list
    home_starters = home_header_list[2:7]
    home_bench = home_header_list[8:-1]
    home_players = [home_starters,home_bench]

    # Consolodate the away team list of lists into one list
    away_starters_list = [item for sublist in away_starters for item in sublist]
    away_bench_list = [item for sublist in away_bench for item in sublist]
    away_roster = away_starters_list + away_bench_list

    # Consolodate the home team list of lists into one list
    home_starters_list = [item for sublist in home_starters for item in sublist]
    home_bench_list = [item for sublist in home_bench for item in sublist]
    home_roster = home_starters_list + home_bench_list

    # Add Team Totals to the end of the away team list
    away_roster.append('Team Totals')
    dataframe_dictionary[Away_code_list[i] + Date_list[i]].index = away_roster

    # Add Team Totals to the end of the home team list
    home_roster.append('Team Totals')
    dataframe_dictionary[Home_code_list[i] + Date_list[i]].index = home_roster

    # Set the home and away dataframe column labels
    column_headers = away_header_list[1]
    column_headers.remove("Starters")
    dataframe_dictionary[Away_code_list[i] + Date_list[i]].columns = column_headers
    dataframe_dictionary[Home_code_list[i] + Date_list[i]].columns = column_headers

    dataframe_dictionary[Away_code_list[i] + Date_list[i]]['Date'] = Actual_date_list[i]
    dataframe_dictionary[Home_code_list[i] + Date_list[i]]['Date'] = Actual_date_list[i]
    
    home_stats_df_list.append(dataframe_dictionary[Home_code_list[i] + Date_list[i]])
    away_stats_df_list.append(dataframe_dictionary[Away_code_list[i] + Date_list[i]])
    
    # Pull logistical information about the game: attendance, length of game, and venue
    text = soup.get_text()
    attendance_string = re.findall(r"Attendance:\s+\d*,\d*", text)
    
    # A failsafe, as one of the games has no attendance listed causing the loop to break
    if(len(attendance_string) != 0):
        
        # Clean up the attendance string that is pulled 
        new_attendance_string = attendance_string[0].strip('([^a-z]xa)')
        attendance_list = list(new_attendance_string)
        attendance_list.pop(11)
    else:
        attendance_string = "No Attendance Available"
    
    # Clean up the game length string that is pulled 
    game_time_string = re.findall(r"Time of Game:\s+\d:\d*", text)    
    new_game_time_string = game_time_string[0].strip('([^a-z]xa)')
    game_time_list = list(new_game_time_string)
    game_time_list.pop(13)

    # join the lists to create final string
    final_attendance_string = "".join(attendance_list)
    final_game_time_sting = "".join(game_time_list)

    # Create throwaway variables to hold split strings so we can pick out what we want later
    x = final_attendance_string.split(":")
    y = final_game_time_sting.split(":",1)

    # Pick out the strings before the : that we split above
    dataframe_dictionary['Attendance' + Date_list[i]] = pd.DataFrame(x, columns = [x[0]])
    dataframe_dictionary['Attendance' + Date_list[i]] = dataframe_dictionary['Attendance' + Date_list[i]].drop([0])
    dataframe_dictionary['Game_Time' + Date_list[i]] = pd.DataFrame(y, columns = [y[0]])
    dataframe_dictionary['Game_Time' + Date_list[i]] = dataframe_dictionary['Game_Time' + Date_list[i]].drop([0])

    # Join the attendance and game time dataframes into one dataframe
    dataframe_dictionary['Game_Logistics' + Date_list[i]] = pd.concat([dataframe_dictionary['Attendance' + Date_list[i]], dataframe_dictionary['Game_Time' + Date_list[i]]], ignore_index=True, sort=False)
    dataframe_dictionary['Game_Logistics' + Date_list[i]]['Time of Game'] = dataframe_dictionary['Game_Logistics' + Date_list[i]]['Time of Game'].shift(-1)
    dataframe_dictionary['Game_Logistics' + Date_list[i]] = dataframe_dictionary['Game_Logistics' + Date_list[i]].dropna()

    # Scrape the name of the vanue and location and add that to the logistics dataframe
    divparent = soup.find('div', attrs={'class':'scorebox_meta'})
    divparent.text
    text = divparent.text
    result = re.split(r'\n', text)
    almost_done = result[1]
    final_result = almost_done.partition(f'{Year_list[i]}')[2]
    dataframe_dictionary['Game_Logistics' + Date_list[i]] = dataframe_dictionary['Game_Logistics' + Date_list[i]].assign(Venue=[final_result])

    dataframe_dictionary['Game_Logistics' + Date_list[i]]['Date'] = Actual_date_list[i]
    game_logistics_df_list.append(dataframe_dictionary['Game_Logistics' + Date_list[i]])
    

In [6]:
########## GUIDE TO ACCESSING DATAFRAMES ##########

## Pull Home Stats --> dataframe_dictionary['(Home Code)(Date)']

## Pull Away Stats --> dataframe_dictionary['(Away Code)(Date)']

## Pull Box Score --> dataframe_dictionary['Box(Date)']

## Pull Game Logistics --> dataframe_dictionary['Game_Logistics(Date)']

In [7]:
#Test
dataframe_dictionary['Game_Logistics201812020']

,Attendance,Time of Game,Venue,Date
0,"19,600",2:14,"AmericanAirlines Arena, Miami, Florida",12/2/2018


In [8]:
#Test
dataframe_dictionary['UTA201812020']

,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,Date
Donovan Mitchell,37:40,8,24,.333,1,7,.143,1,1,1.000,...,5,5,3,1,0,2,2,18,-5,12/2/2018
Rudy Gobert,37:15,5,9,.556,0,0,,2,5,.400,...,14,18,4,1,3,2,2,12,+19,12/2/2018
Ricky Rubio,36:05,9,13,.692,2,4,.500,3,3,1.000,...,5,5,6,1,0,3,3,23,+14,12/2/2018
Jae Crowder,34:09,6,15,.400,3,7,.429,4,4,1.000,...,4,6,2,1,0,4,3,19,+1,12/2/2018
Joe Ingles,29:10,5,13,.385,1,4,.250,0,0,,...,4,4,4,2,0,0,4,11,+7,12/2/2018
Royce O'Neale,18:43,2,2,1.000,2,2,1.000,0,0,,...,2,2,1,0,1,1,1,6,+4,12/2/2018
Derrick Favors,17:53,2,4,.500,1,2,.500,0,0,,...,4,7,0,2,2,0,2,5,-19,12/2/2018
Kyle Korver,17:23,1,6,.167,1,5,.200,0,0,,...,1,2,0,0,0,1,0,3,-18,12/2/2018
Thabo Sefolosha,6:39,0,1,.000,0,1,.000,0,0,,...,1,1,0,0,0,0,0,0,-5,12/2/2018
Raul Neto,3:07,1,1,1.000,0,0,,0,0,,...,0,0,1,0,0,0,0,2,-4,12/2/2018


In [9]:
#Test
dataframe_dictionary['MIA201812020']

,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,Date
Josh Richardson,39:10,4,16,.250,2,6,.333,6,6,1.000,...,2,3,4,0,0,4,2,16,-13,12/2/2018
Wayne Ellington,37:12,4,13,.308,4,9,.444,0,0,,...,5,7,1,1,0,0,4,12,-2,12/2/2018
Hassan Whiteside,34:02,11,17,.647,0,0,,1,6,.167,...,13,20,2,1,3,1,4,23,-8,12/2/2018
James Johnson,18:46,4,10,.400,1,2,.500,0,0,,...,1,2,2,1,1,2,2,9,-16,12/2/2018
Rodney McGruder,5:07,0,1,.000,0,0,,0,0,,...,0,0,0,0,0,0,0,0,-14,12/2/2018
Justise Winslow,32:05,4,9,.444,1,2,.500,0,0,,...,5,5,1,2,1,4,3,9,+14,12/2/2018
Dwyane Wade,30:26,4,15,.267,2,6,.333,5,5,1.000,...,3,3,8,1,1,1,0,15,+21,12/2/2018
Kelly Olynyk,29:10,4,8,.500,3,6,.500,5,7,.714,...,6,8,1,1,1,1,0,16,+18,12/2/2018
Bam Adebayo,14:01,1,2,.500,0,0,,0,0,,...,1,2,1,1,1,0,1,2,+10,12/2/2018
Tyler Johnson,Did Not Dress,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,12/2/2018


In [10]:
#Test
dataframe_dictionary['Box201812020']

,1Q,2Q,3Q,4Q,T,Date
UTA,35,20,22,23,100,12/2/2018
MIA,27,26,27,22,102,12/2/2018


In [27]:
total_box_score_df = pd.concat(box_score_df_list)

# column_list = total_box_score_df.columns.tolist()
# date_holder = 'Date'
# column_list.remove(date_holder)
# column_list.insert(0, date_holder)
# total_box_score_df = total_box_score_df[column_list]

total_box_score_df

,1Q,2Q,3Q,4Q,T,Date,1OT
UTA,16,32,20,28,96,2/11/2014,NaN
LAL,27,10,23,19,79,2/11/2014,NaN
ATL,31,26,25,16,98,1/2/2015,NaN
UTA,21,24,23,24,92,1/2/2015,NaN
MEM,30,26,29,22,107,4/3/2016,NaN
ORL,29,38,30,22,119,4/3/2016,NaN
UTA,24,23,22,32,101,12/27/2017,NaN
GSW,23,25,42,36,126,12/27/2017,NaN
LAL,31,22,28,36,117,11/17/2018,NaN
ORL,25,37,38,30,130,11/17/2018,NaN


In [12]:
total_home_stats_df = pd.concat(home_stats_df_list)
total_home_stats_df

,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,Date
Wesley Johnson,37:21,7,14,.500,1,3,.333,0,0,,...,6,6,0,0,0,2,3,15,-14,2/11/2014
Chris Kaman,36:44,11,24,.458,0,0,,3,3,1.000,...,10,14,4,1,3,3,3,25,-5,2/11/2014
Steve Blake,36:25,2,11,.182,1,4,.250,0,0,,...,3,5,8,2,0,1,2,5,-3,2/11/2014
Shawne Williams,36:13,4,8,.500,2,4,.500,1,2,.500,...,5,6,0,0,2,0,3,11,-10,2/11/2014
Steve Nash,16:35,1,4,.250,0,2,.000,0,0,,...,1,1,2,0,0,0,0,2,-6,2/11/2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Alex Len,0:02,0,0,,0,0,,0,0,,...,0,0,0,0,0,0,0,0,0,1/21/2023
PJ Dozier,Did Not Play,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,1/21/2023
Matthew Dellavedova,Did Not Play,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,1/21/2023
Terence Davis,Did Not Play,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,1/21/2023


In [13]:
total_away_stats_df = pd.concat(away_stats_df_list)
total_away_stats_df

,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,Date
Gordon Hayward,34:46,5,13,.385,2,4,.500,3,4,.750,...,8,8,7,2,0,4,2,15,+12,2/11/2014
Trey Burke,28:49,3,11,.273,1,3,.333,1,1,1.000,...,3,4,8,1,0,1,1,8,+2,2/11/2014
Derrick Favors,27:03,4,9,.444,0,0,,3,7,.429,...,8,10,1,1,3,1,3,11,-4,2/11/2014
Marvin Williams,22:29,4,8,.500,1,2,.500,0,0,,...,4,7,1,0,0,1,0,9,-4,2/11/2014
Richard Jefferson,19:53,0,4,.000,0,2,.000,2,2,1.000,...,1,1,2,0,0,0,0,2,-13,2/11/2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Danuel House Jr.,5:28,2,3,.667,2,2,1.000,0,0,,...,0,0,0,0,0,0,0,6,+10,1/21/2023
Furkan Korkmaz,Did Not Play,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,1/21/2023
Joel Embiid,Did Not Dress,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,1/21/2023
Jaden Springer,Did Not Play,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,1/21/2023


In [14]:
total_game_logistics_df = pd.concat(game_logistics_df_list)
total_game_logistics_df

,Attendance,Time of Game,Venue,Date
0,"18,209",1:59,"STAPLES Center, Los Angeles, California",2/11/2014
0,"19,029",2:09,"EnergySolutions Arena, Salt Lake City, Utah",1/2/2015
0,"17,741",2:08,"Amway Center, Orlando, Florida",4/3/2016
0,"19,596",2:11,"Oracle Arena, Oakland, California",12/27/2017
0,"19,249",2:10,"Amway Center, Orlando, Florida",11/17/2018
0,"15,898",2:19,"Amway Center, Orlando, Florida",11/18/2018
0,"19,600",2:14,"AmericanAirlines Arena, Miami, Florida",12/2/2018
0,"16,447",2:06,"State Farm Arena, Atlanta, Georgia",11/8/2019
0,"16,447",2:10,"Golden 1 Center, Sacramento, California",11/3/2021
0,"12,993",2:15,"Golden 1 Center, Sacramento, California",11/7/2021


In [28]:
index_list = total_box_score_df.index.tolist()

for i in range(len(index_list)):
    if (i % 2) == 0:
        index_list[i] = 'A-' + index_list[i]
    else:
        index_list[i] = 'H-' + index_list[i]

total_box_score_df = total_box_score_df.set_index('Date')

total_box_score_df.insert(0, 'Team', index_list)
total_box_score_df

,Team,1Q,2Q,3Q,4Q,T,1OT
Date,,,,,,,
2/11/2014,A-UTA,16,32,20,28,96,NaN
2/11/2014,H-LAL,27,10,23,19,79,NaN
1/2/2015,A-ATL,31,26,25,16,98,NaN
1/2/2015,H-UTA,21,24,23,24,92,NaN
4/3/2016,A-MEM,30,26,29,22,107,NaN
4/3/2016,H-ORL,29,38,30,22,119,NaN
12/27/2017,A-UTA,24,23,22,32,101,NaN
12/27/2017,H-GSW,23,25,42,36,126,NaN
11/17/2018,A-LAL,31,22,28,36,117,NaN


In [30]:
total_game_logistics_df.to_csv('total_game_logistics.csv')
total_away_stats_df.to_csv('total_away_stats.csv')
total_home_stats_df.to_csv('total_home_stats.csv')
total_box_score_df.to_csv('total_box_score_df.csv')

In [25]:
# total_box_score_df

#df.set_index(list1)

# Use this if you wanna assing one of the existing DataFrame columns as Index


,1Q,2Q,3Q,4Q,T,1OT
Date,,,,,,
2/11/2014,16,32,20,28,96,NaN
2/11/2014,27,10,23,19,79,NaN
1/2/2015,31,26,25,16,98,NaN
1/2/2015,21,24,23,24,92,NaN
4/3/2016,30,26,29,22,107,NaN
4/3/2016,29,38,30,22,119,NaN
12/27/2017,24,23,22,32,101,NaN
12/27/2017,23,25,42,36,126,NaN
11/17/2018,31,22,28,36,117,NaN


TypeError: 'list' object is not callable

['A-UTA',
 'H-LAL',
 'A-ATL',
 'H-UTA',
 'A-MEM',
 'H-ORL',
 'A-UTA',
 'H-GSW',
 'A-LAL',
 'H-ORL',
 'A-NYK',
 'H-ORL',
 'A-UTA',
 'H-MIA',
 'A-SAC',
 'H-ATL',
 'A-NOP',
 'H-SAC',
 'A-IND',
 'H-SAC',
 'A-TOR',
 'H-SAC',
 'A-UTA',
 'H-SAC',
 'A-SAS',
 'H-SAC',
 'A-HOU',
 'H-SAC',
 'A-BOS',
 'H-SAC',
 'A-PHO',
 'H-SAC',
 'A-NOP',
 'H-POR',
 'A-LAC',
 'H-SAC',
 'A-MEM',
 'H-SAC',
 'A-CHO',
 'H-SAC',
 'A-OKC',
 'H-SAC',
 'A-PHI',
 'H-SAC']